In [73]:
import pandas as pd
import psycopg2
from psycopg2.extras import NamedTupleCursor

conn = psycopg2.connect(database='finance', cursor_factory=NamedTupleCursor)

with conn.cursor() as cur:
    cur.execute('''
    WITH inflows AS (SELECT SUM(total) FROM transactions WHERE target = 'Cash'),
        outflows AS (SELECT SUM(total) FROM transactions WHERE source = 'Cash')
    SELECT inflows.sum - outflows.sum AS cash FROM inflows CROSS JOIN outflows;
    ''')
    cash = cur.fetchone().cash
    
with conn.cursor() as cur:
    cur.execute('''
    SELECT target AS ticker, SUM(units) FROM transactions WHERE txtype = 'buy' GROUP BY ticker ORDER BY ticker ASC;
    ''')
    units = cur.fetchall()
    
tickers = '+'.join([x.ticker for x in units])
prices = pd.read_csv('http://finance.yahoo.com/d/quotes.csv?s=' + tickers + '&f=sbp', header=None)
prices.set_index(0, inplace=True)  

from decimal import Decimal
previous = cash
total = cash
for asset in units:
    asset_value = asset.sum * Decimal(prices.ix[asset.ticker][1])
    prev_asset_value = asset.sum * Decimal(prices.ix[asset.ticker][2])
    total += asset_value
    previous += prev_asset_value
    day = asset_value - prev_asset_value
    pctg = 100 * day / previous
    print(f"{asset.ticker:6}: {asset_value:10,.2f}  ({day:+8,.2f}, {pctg:+5.2f}%)")

day = total - previous
pctg = 100 * day / previous
print(f"Cash  : {cash:10,.2f}")
print(f"Total : {total:10,.2f}  ({day:+8,.2f}, {pctg:+5.2f}%)")

VAB.TO:   2,165.80  (  +11.05, +0.51%)
VCN.TO:   2,173.50  (   +5.52, +0.13%)
VEE.TO:   2,183.76  (  +13.68, +0.21%)
VI.TO :   2,143.68  (   -0.84, -0.01%)
VIU.TO:   4,385.92  (   +8.80, +0.07%)
VRE.TO:   2,186.70  (  +22.94, +0.15%)
VUN.TO:   6,554.95  (   +0.00, +0.00%)
Cash  :      21.37
Total :  21,815.68  (  +61.15, +0.28%)
